In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git

Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 123 (delta 42), reused 118 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (123/123), 141.67 KiB | 2.72 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls

/kaggle/working/NIDS_Coursework/My_Code
classification.py    main.ipynb     record_level_embedding.py
featureSelection.py  preprocess.py  transformer.py


In [3]:
%rm -r __pycache__

rm: cannot remove '__pycache__': No such file or directory


In [4]:
!git pull origin main

From https://github.com/Tanveer2719/NIDS_Coursework
 * branch            main       -> FETCH_HEAD
Already up to date.


In [5]:
import pandas as pd
import numpy as np

path = "/kaggle/input/unsw-nb-15/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')

print(df.dtypes)


(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean   

In [6]:
# perform preprocessing

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)
processed_features.head()

Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,False
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,False
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,False
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,False
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)
processed_full_df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11,label
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,0
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,0
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,0
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,0
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,0


In [10]:
from featureSelection import KBestFeatureSelector

kbest = KBestFeatureSelector(k=50)  # Set k to your desired number of features
selected_mask, selected_feature = kbest.select_features(df=processed_full_df, target_column='label', verbose=True)

selected_feature


🔍 Starting SelectKBest feature selection (k = 50) using mutual_info_classif...
✅ Selected top 50 features out of 95
📌 Selected features:
   1. dur
   2. spkts
   3. dpkts
   4. sbytes
   5. dbytes
   6. rate
   7. sttl
   8. dttl
   9. sload
  10. dload
  11. sloss
  12. dloss
  13. sinpkt
  14. dinpkt
  15. sjit
  16. djit
  17. swin
  18. stcpb
  19. dtcpb
  20. dwin
  21. tcprtt
  22. synack
  23. ackdat
  24. smean
  25. dmean
  26. response_body_len
  27. ct_srv_src
  28. ct_state_ttl
  29. ct_dst_ltm
  30. ct_src_dport_ltm
  31. ct_dst_sport_ltm
  32. ct_dst_src_ltm
  33. is_ftp_login
  34. ct_src_ltm
  35. ct_srv_dst
  36. is_sm_ips_ports
  37. proto_1
  38. proto_2
  39. proto_3
  40. proto_4
  41. proto_5
  42. proto_6
  43. service_1
  44. service_2
  45. service_5
  46. service_8
  47. state_1
  48. state_2
  49. state_3
  50. state_4


Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit',
       'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean',
       'dmean', 'response_body_len', 'ct_srv_src', 'ct_state_ttl',
       'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports',
       'proto_1', 'proto_2', 'proto_3', 'proto_4', 'proto_5', 'proto_6',
       'service_1', 'service_2', 'service_5', 'service_8', 'state_1',
       'state_2', 'state_3', 'state_4'],
      dtype='object')

In [19]:
columns_to_keep = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit',
       'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean',
       'dmean', 'response_body_len', 'ct_srv_src', 'ct_state_ttl',
       'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports',
       'proto_1', 'proto_2', 'proto_3', 'proto_4', 'proto_5', 'proto_6',
       'service_1', 'service_2', 'service_5', 'service_8', 'state_1',
       'state_2', 'state_3', 'state_4']

In [20]:
df_selected = processed_full_df[columns_to_keep].copy()
df_selected.shape

(257673, 50)

In [21]:
df_selected.dtypes

dur                  float64
spkts                float64
dpkts                float64
sbytes               float64
dbytes               float64
rate                 float64
sttl                 float64
dttl                 float64
sload                float64
dload                float64
sloss                float64
dloss                float64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                 float64
stcpb                float64
dtcpb                float64
dwin                 float64
tcprtt               float64
synack               float64
ackdat               float64
smean                float64
dmean                float64
response_body_len    float64
ct_srv_src           float64
ct_state_ttl         float64
ct_dst_ltm           float64
ct_src_dport_ltm     float64
ct_dst_sport_ltm     float64
ct_dst_src_ltm       float64
is_ftp_login         float64
ct_src_ltm           float64
ct_srv_dst    

In [22]:
numerical_columns = [col for col in columns_to_keep 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_"))]


In [23]:
from record_level_embedding import RecordLevelEmbedder

embedder = RecordLevelEmbedder(selected_df = df_selected, numerical_columns = numerical_columns, embed_dimension= 64)

embedded_df = embedder.transform_to_df()

embedded_df.head()

[INFO] Initializing RecordLevelEmbedder...
[INFO] Detecting categorical blocks...
[INFO] Detected 3 categorical blocks:
   - proto: 6 columns
   - service: 4 columns
   - state: 4 columns
[INFO] Embedder initialized with embedding dimension 64.
[INFO] Building Keras embedding model...
   - Adding numerical input: dur
   - Adding numerical input: spkts
   - Adding numerical input: dpkts
   - Adding numerical input: sbytes
   - Adding numerical input: dbytes
   - Adding numerical input: rate
   - Adding numerical input: sttl
   - Adding numerical input: dttl
   - Adding numerical input: sload
   - Adding numerical input: dload
   - Adding numerical input: sloss
   - Adding numerical input: dloss
   - Adding numerical input: sinpkt
   - Adding numerical input: dinpkt
   - Adding numerical input: sjit
   - Adding numerical input: djit
   - Adding numerical input: swin
   - Adding numerical input: stcpb
   - Adding numerical input: dtcpb
   - Adding numerical input: dwin
   - Adding numeric

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_54,embed_55,embed_56,embed_57,embed_58,embed_59,embed_60,embed_61,embed_62,embed_63
0,0.215637,0.051863,0.001615,-0.666279,-0.485952,0.200932,0.072335,0.427739,-0.004813,-0.440847,...,0.525813,0.764477,-0.288543,0.186886,-0.056883,0.248672,-0.065912,0.545021,0.113077,-0.529118
1,0.179579,0.443367,0.336470,-0.657083,-0.649067,0.133751,0.180447,0.598313,0.003251,-0.548584,...,0.338089,0.668407,-0.069773,0.334233,0.066996,0.123631,0.038619,0.282547,0.045118,-0.673972
2,0.230335,0.275807,0.144161,-0.652496,-0.585472,0.125730,0.074760,0.579503,0.042990,-0.459546,...,0.223260,0.813105,-0.175390,0.243983,0.053493,0.064585,0.060357,0.440102,-0.068346,-0.791642
3,0.294484,-0.012795,0.042549,-0.418629,-0.787162,0.123961,0.135255,0.492020,0.110328,-0.380874,...,0.211627,0.559256,-0.411054,0.088590,-0.118838,0.155792,-0.227468,0.324657,0.174132,-0.774244
4,-0.052648,0.566663,0.265931,-0.334916,-0.272101,-0.022786,-0.268528,0.828640,0.056920,-0.497824,...,0.207685,0.759384,-0.207660,0.556303,-0.006968,-0.205409,-0.104363,0.665722,0.104833,-0.643198


In [24]:
# Binary classification
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model


print(processed_full_df['label'].unique())

binary_target = (processed_full_df['label'] != 0).astype(int).values
unique, counts = np.unique(binary_target, return_counts=True)
print("Classes and their counts:")
for cls, cnt in zip(unique, counts):
    print(f"Class {cls}: {cnt}")

[0 1]
Classes and their counts:
Class 0: 93000
Class 1: 164673


In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from transformer import BasicTransformer
from classification import LastTokenClassificationHead
from sklearn.model_selection import train_test_split



# input data
X_embed = embedded_df.values                   # shape: (num_samples, embed_dim)
X_embed_seq = tf.expand_dims(X_embed, axis=1)  # shape: (batch_size, 1, embed_dim)

input_layer = Input(shape=(1, X_embed.shape[1]), name="record_input")

# Create transformer
transformer = BasicTransformer(
    n_layers=2,
    internal_size=128,
    n_heads=4,
    verbose=True  # See progress!
)


x = transformer.apply(input_layer, training=True)


# Create head
classification_head = LastTokenClassificationHead()

# Pool last token
x = classification_head.apply(x)

#  Add MLP head  
for i, layer_size in enumerate([128]):
    x = Dense(layer_size, activation="relu", name=f"classification_mlp_{i}_{layer_size}")(x)
    x = Dropout(0.1)(x)

#  output layer for binary classification
output_layer = Dense(1, activation="sigmoid", name="binary_output")(x)

# Build model  
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile model 
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
    jit_compile=True
)

# split data

X_np = X_embed_seq.numpy() if hasattr(X_embed_seq, 'numpy') else np.array(X_embed_seq)


X_train, X_val, y_train, y_val = train_test_split(
    X_np, binary_target, test_size=0.2, random_state=42, stratify=binary_target
)

# Train
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

[BasicTransformer] Applying Encoder Block 1/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Applying Encoder Block 2/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Final output shape: (None, 1, 64)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ record_input (InputLayer)       │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_0_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_1_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ slice_last (Lambda)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_mlp_0_128        │ (None, 128)            │         8,320 │
│ (Dense)                         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ binary_output (Dense)           │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 307,457 (1.17 MB)

 Trainable params: 307,457 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9004 - loss: 0.1982[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 43s 5ms/step - accuracy: 0.9004 - loss: 0.1982 - val_accuracy: 0.9135 - val_loss: 0.1721
Epoch 2/20
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9122 - loss: 0.1739 - val_accuracy: 0.9243 - val_loss: 0.1595
Epoch 3/20
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9180 - loss: 0.1647 - val_accuracy: 0.9199 - val_loss: 0.1602
Epoch 4/20
6442/6442 

In [26]:
# evaluate the model
results = model.evaluate(X_val, y_val)
print(f"Validation loss: {results[0]:.4f}, Validation accuracy: {results[1]:.4f}")


1611/1611 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9338 - loss: 0.1347
Validation loss: 0.1347, Validation accuracy: 0.9348


In [27]:
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Predict probabilities
y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob >= 0.5).astype(int).flatten()

# F1
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
print("Confusion Matrix:\n", cm)

if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    print(f"Detection Rate (Recall): {detection_rate:.4f}")
    print(f"False Alarm Rate: {false_alarm_rate:.4f}")
else:
    print("Confusion matrix does not contain both classes!")
    print("Check your labels: y_val:", np.unique(y_val), " y_pred:", np.unique(y_pred))


[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
1602/1611 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
F1 Score: 0.9489
Confusion Matrix:
 [[16987  1613]
 [ 1748 31187]]
Detection Rate (Recall): 0.9469
False Alarm Rate: 0.0867
